![Logo](https://github.com/Columbia-Neuropythonistas/IntroPythonForNeuroscientists2023/assets/65978061/138766b5-ac36-4dc8-b9d4-bf512ecebe78)

# **Week 8: Data Visualization and Object-Oriented Programming**

## Data Visualization first, with *real data!*
We'll begin today by loading in some (real!) data. Before we do that though, we need to import some packages.

In [ ]:
#remember that when we use 'as', we are simply telling Python what term we want to refer to our imported packages by
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#this does some important stuff behind the scenes for plotting specifically for jupyter notebooks (same in vscode), ask us if you want to learn more!
%matplotlib inline 


Now, let's load and examine our data!

In [ ]:
data = pd.read_csv('SST_data.csv')
data

This dataset is borrowed from [Neuromatch Academy](https://compneuro.neuromatch.io/projects/neurons/README.html), but this is very much real data, generated by the Allen Institute! Briefly, what you see here are two-photon calcium imaging signals from a single mouse performing a visual change detection task. I've curated this dataset just a little bit, so we're only looking at SST-expressing interneurons. To better understand what's going on here, let's traverse this dataframe a little bit.

P.S. If you want to learn more about the dataset, check out the youtube video in the NMA link!

In [ ]:
#You can index dataframes using .COLUMN_NAME or by using brackets: [COLUMN_NAME] - Pandas is very flexible!
data.cell_id.unique()

In [ ]:
singlecell_trial_data = data[(data.trial_id == 24) & (data.cell_id == 1086500633)]
singlecell_trial_data

Ok, now that we've examined the dataset a little bit, let me provide some documentation:

`dF/F` is the instantaneous calcium imaging signal <br>
`time_from_stim` is the timepoint of each row of data, aligned to an image presentation <br>
`cell_id` self explanatory I hope <br>
`exposure` whether the image for a a trial was familiar or novel  <br>
`trial_id` each image presentation is a separate trial <br>
`omitted` whether a trial had an omitted image <br>
`pupil_area` measured 500ms after stimulus presntation <br>
`mean_response` average dF/F over the 500ms following image presentation <br>

### When do I use matplotlib, and when do I use seaborn?

Think of matplotlib as a basic set of tools with which to build plots in Python. Often times, we just quickly want to visualize data, which could be as simple as an array/vector of points. Matplotlib is amazing at this.

In [ ]:
single_trial_data = data[(data.trial_id == 605) & (data.cell_id == 1086500092)]
single_trial_trace = np.array(single_trial_data['dF/F'])
single_trial_trace

In [ ]:
plt.plot(single_trial_trace)

What other quick functionality might we want here?

In [ ]:
single_trial_timepoints = np.array(single_trial_data['time_from_stim'])
plt.plot(single_trial_timepoints, single_trial_trace)

Ok, this is great, but we have tons of data! What if we want to plot that?

In [ ]:
plt.plot(data['dF/F'])

That isn't very useful ... We could go through and organize our dataframe, take some averages, and then do a lot of work to make the plot look pretty, but it would be way easier if something did that automatically ...

## Plotting data with Seaborn

One thing that's important to note about our dataset is that it's long-form, not wide (e.g. column labels don't correspond to time, but rather, each row represents a separate single observation). While this may seem confusing at first, it makes things intensely convenient when we use **Seaborn**, a plotting package that's built on top of matplotlib.

In [ ]:
#ignore this for now, it just makes things look good
sns.set_style("ticks")
sns.set_context("notebook")

sns.lineplot(data = data, x = 'time_from_stim', y = 'dF/F')

Seaborn is super simple to use! We pass in a dataframe, and then specify what we want to plot on the x-axis, and what we want to plot on the y-axis, using labels from the dataframe.

What if we want our x-axis to be categorical instead? Let's google a solution! We can use **documentation** to understand how to use Seaborn. [Let's go!](https://seaborn.pydata.org/)

## Problem 1
Make a barplot, but instead of using mean response data, plot pupil size instead. As a challenge, try to adjust the y-axis to bring out the difference between familiar and novel. Use google to find a solution!

## Splitting up data in Seaborn
What if we want to split up our lineplot by familiar and novel trials? Seaborn makes this super easy by allowing you to pass a label to `hue`.

In [ ]:
sns.lineplot(data = data, x = 'time_from_stim', y = 'dF/F', hue = 'exposure')

Notice how seaborn always includes error intervals? We can turn thes off if we'd like, but let's leave them be for now. Let's now check the documentation to find other ways to split up data.

In [ ]:
sns.lineplot(data = data, x = 'time_from_stim' , y = 'dF/F', hue = 'exposure', style = 'omitted')

## Problem 2
1) Can you figure out how to make a histogram of pupil area in Seaborn? Here's some edited data for you to use. <br>
2) Once you do this, can you figure out how to split the histogram by `exposure`? <br>
3) CHALLENGE: If you have more time, play around with your plot! Be creative and see what else you can add to your histogram, using the documentation as a guide.

In [ ]:
data_sample = data.sample(1000)

### Saving Figures
Saving figures can be tricky at times. You have to specify a high DPI (>100, depends on the size of your plot) if you want decent resolution, and sometimes axes can get lopped off.

In [ ]:
plt.figure(figsize=(8,4))
sns.lineplot(data = data, x = 'time_from_stim' , y = 'dF/F', hue = 'exposure', style = 'omitted')
plt.xlabel('Delta Stim')
plt.ylabel('dF/F')
plt.title('Real Data')

plt.savefig('my_figure.png', dpi = 300)

Oh no! Our axes, it's broken!

In [ ]:
plt.figure(figsize=(8,4))
sns.lineplot(data = data, x = 'time_from_stim' , y = 'dF/F', hue = 'exposure', style = 'omitted')
plt.xlabel('Delta Stim')
plt.ylabel('dF/F')
plt.title('Real Data')
#use this bbox_inches command to fix things
plt.savefig('my_figure.png', dpi = 300, bbox_inches = 'tight')

## Object-Oriented Programming (OOP)

**Object-oriented programming** is a style of programming that is used heavily in Python packages. To understand waht it is, let's understand what it's not: **functional programming**.

In [ ]:
#imports
from datetime import date
from abc import ABC, abstractmethod
import numpy as np
import pandas as pd

This is functional-style code - take just a few minutes to guess what this will output.

In [ ]:
abhi = {'name': 'Abhi',
        'units': [6,7,8],
        'cblind': True}

sharon = {'name': 'Sharon',
        'units': [1,2],
        'cblind': False}

def print_teacher_info(teacher):
    if teacher['cblind']:
        cblind = 'is'
    else:
        cblind = 'is NOT'
    print(teacher['name'] + ' is teaching units ' + str(teacher['units']) + ' and ' + cblind + ' colorblind!')
    
print_teacher_info(abhi)
print_teacher_info(sharon)

In [ ]:
#ENTER YOUR GUESS HERE, AND THEN RUN THE CODE ABOVE TO CHECK

Now, let's look at an OOP example of the same code.

In [ ]:
class Teacher:
    
    def __init__(self, name, units, colorblind):
        self.name = name
        self.units = units
        self.colorblind = colorblind
        
    def print_teacher_info(self):
        if self.colorblind:
            cblind = 'is'
        else:
            cblind = 'is NOT'
        print(self.name + ' is teaching units ' + str(self.units) + ' and ' + cblind + ' colorblind!')
        
abhi = Teacher('Abhi', [7,8,9], True)
sam = Teacher('Sam', [4,5,6], False)
abhi.print_teacher_info()
sam.print_teacher_info()

#### What is OOP?
At it's root, OOP is about encapsulation and modularity. We'll go over the specifics in the lesson today!


As you can see, it is possible to use both functional and OOP coding styles in Python. Today, we're going to go over exactly what OOP is, what it's useful for, and how to read OOP code.

### Classes and objexts
We've actually already used OOP before! For example, does the code below look familiar?

In [ ]:
df = pd.DataFrame([1,2,3])

In this code, df is an **object**, defined in the DatFrame **class**. Confusing?

Let's use an analogy: the class is a recipe, and the object is the food you make using that recipe.
In fact, almost everything in Python is an object. For example:

In [ ]:
a = 3.1
b = int(3)
print(type(a))
print(type(b))

See how it says class? What do we think type (df) will return?

In [ ]:
type(df)

#### Example class code
Now let's go over how to use classes and objects in detail. Run the code below to load in our custom `Experiment` class. Remember that this class is like a recipe to make specific objects!

In [ ]:
#Run this code to load the class
class Experiment:
        
    def __init__(self, path_to_expt, expt_date, experimenter):
        self.path_to_expt = path_to_expt
        self.expt_date = expt_date
        self.experimenter = experimenter
        self.generated_date = date.today()
        print('Constructor called')
        
    def print_expt_info(self):
        print('Path: ', self.path_to_expt)
        print('Experiment Date: ', self.expt_date)
        print('Experimenter: ', self.experimenter)
        print('Generation Date: ', self.generated_date)
        
    def return_data(self):
        return 'There is no data here for now'

Let's pause to go through what's inside the class code. 
First, we have a function called `__init__`. This is a **constructor**, and it will be run anytime you create an object from this class. The constructor is a place to put commands that you create an object: for example, here we assign some attributes (or variables) associated with out class.

Let's see the constructor in action by creating an object First, we create an object named `expt`, by calling `Experiment` and providing information. This process is called **instantiation**.

In [ ]:
expt = Experiment('experiment.csv', '342321', 'Abhi')

What happened here is we created an object called `expt` from Experiment, which automatically ran the constructor. Notice that when we instantiated our object, we provided information to the function call, just like you would with any other function. We can access this data, as in our constructor we save the data to the object using the **self** command.

In [ ]:
expt.expt_date

We've already seen attributes before - can you think of an example?

In [ ]:
np.array([1,2,3]).shape


We can also call functions within the class. For example:

In [ ]:
expt.print_expt_info()

What happens if we try to call the function directly?

In [ ]:
#print_expt_info()
Experiment.print_expt_info()

In OOP, self refers to the *object itself*. That is to say: you can't call a function without an object. We can get a little hacky and pass in the object though.

In [ ]:
Experiment.print_expt_info(self=expt)

This is what happens when you call functions from a class - it just happens to pass `self` (a reference to the object) in for you!

### Problem 3

Modify the code from the cells above to add another argument to the constructor called `results`. Then, create an object of your class, and call  `print_expt_info`.

In [ ]:
#Edit this code to add an argument to the constructor, and then create an object from the class that utilizes the new argument.
class Experiment:
        
    #HI, I AM THE CONSTRUCTOR
    def __init__(self, path_to_expt, expt_date, experimenter):
        self.path_to_expt = path_to_expt
        self.expt_date = expt_date
        self.experimenter = experimenter
        self.generated_date = date.today()
        print('Constructor called')
        
    def print_expt_info(self):
        print('Path: ', self.path_to_expt)
        print('Experiment Date: ', self.expt_date)
        print('Experimenter: ', self.experimenter)
        print('Generation Date: ', self.generated_date)
        
    def return_data(self):
        return 'There is no data here for now'

#### Making multiple objects
A class can support many independent objects! Back to the analogy: if I have two recipes for a pumpkin pie, I can make two pumpkin pies, and if I put whipped cream on one, then it won't magically appear on the other.

Let's make two objects from a new class, and see if modifying one affects the other. |

In [ ]:
class BehaviorExperiment:
    
    def __init__(self, head_turn, freezing):
        self.head_turn = head_turn
        self.freezing = freezing
        self.time = date.today()
        
    def print_info(self):
        print(self.head_turn)
        print(self.freezing)
        print(self.time)
        
    def calc_velocity(self):
        self.velocity = self.head_turn * 2
        return self.velocity

In [ ]:
beh_expt1 = BehaviorExperiment(5, True)
beh_expt1.print_info()


In [ ]:
velocity = beh_expt1.calc_velocity()
print(velocity)

In [ ]:
beh_expt2 = BehaviorExperiment(20, False)
beh_expt1.print_info()
beh_expt2.print_info()

### Inheritance and polymorphism

As I mentioned earlier, one of the important features of OOP is modularity. Let's go back to the recipe analogy I mentioned earlier. Say we had a recipe for cooking a cake, in general. What if we wanted to bake a vanilla cake? I could write a totally new recipe, but that would be redundant. Instead, what I could do is simply change the section where I add flavorings to the cake mix.

Inheritance is exactly this concept: you can create child classes that inherit from a parent class. Let's see what this means using an example.

In [ ]:
#Run this code to load the class
class Experiment:
        
    def __init__(self, path_to_expt, expt_date, experimenter):
        self.path_to_expt = path_to_expt
        self.expt_date = expt_date
        self.experimenter = experimenter
        self.generated_date = date.today()
        print('Constructor called')
        
    def print_expt_info(self):
        print('Path: ', self.path_to_expt)
        print('Experiment Date: ', self.expt_date)
        print('Experimenter: ', self.experimenter)
        print('Generation Date: ', self.generated_date)
        
    def return_data(self):
        return 'There is no data here for now'

Now, we have a small child class that **inherits** from and **extends** a parent class. Notice the syntax: we just place the name of the parent class in the parenthes at the beginning of the class.

In [ ]:
class ImagingExperiment(Experiment):
    
    def __init__(self, path_to_expt, expt_date, experimenter, frame_rate):
        self.frame_rate = frame_rate
        #Super refers to our parent class
        print('Imaging constructor called')
        super().__init__(path_to_expt, expt_date, experimenter)
    
    #This is a new function!
    def print_frame_rate(self):
        print('Frame Rate: {} Hz'.format(self.frame_rate))
    
    #This is an old function we modified!
    def return_data(self):
        return 'Pretend that I am imaging data' 

Let's start by creating an object of our new class: anyone remember how to do this?

In [ ]:
imaging_expt = ImagingExperiment('experiment_file.csv', '031122', 'Abhi', 30)

A few things to unpack: <br>
1) Notice how we are providing one more argument to the constructor. Let's follow this number.
2) See how the imaging constructor is called first, and then the constructor for the parent experiment class?
3) What do we think the type of our new object will be?

In [ ]:
type(imaging_expt)

Now, let's understand these new functions.

In [ ]:
imaging_expt.print_frame_rate()

That seems self-explanatory - that's a new function we added. Do the old ones still work?

In [ ]:
imaging_expt.print_expt_info()

Ok, what about return_data? What do we think it will output?

In [ ]:
imaging_expt.return_data()

See how we've created a new version of `return_data`? This is called polymorphism - a single function can take many forms in OOP. This is useful, because often you want a child class to subtly modify or add to a parent class. Think about a vegan cake - the general steps might be the same, but you'd want to go back and modify some of the tasks you're peforming to include different ingredients.

### Problem 4
Just as we did with ImagingExperiment, create a class called BehaviorExperiment that inherits from Experiment. In this class, please take in a `behavior_task` variable instead of `frame_rate`. Create a new function in lieu of `frame_rate` to print your `behavior_task`.  In addition, please write a modified `return_data` function to print your behavior task. Use the templates above and don't be afraid of copying and pasting!